[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ryanfobel/gridwatch-history/main)

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0, ".")
sys.path.insert(0, "..")

# Plotting preferences
from cycler import cycler
from matplotlib import rcParams

%matplotlib inline

LIGHT_COLORMAP = [
    (0.533, 0.741, 0.902),
    (0.984, 0.698, 0.345),
    (0.565, 0.804, 0.592),
    (0.965, 0.667, 0.788),
    (0.749, 0.647, 0.329),
    (0.737, 0.6, 0.78),
    (0.929, 0.867, 0.275),
    (0.941, 0.494, 0.431),
    (0.549, 0.549, 0.549),
]

# Plotting preferences
plt.rc("axes", prop_cycle=cycler("color", LIGHT_COLORMAP))
plt.rc("figure", figsize=(12, 6))
#rcParams.update({"figure.figsize": (12, 6)})
bin_width = 0.9
alpha = 0.5
transparent = False
bbox_inches = "tight"
facecolor = "white"

In [2]:
from gridwatch_scraper import load_file

df_plants = load_file("plants.csv", "raw", source="local")
df_capability = load_file("capability.csv", source="local")
df_output = load_file("output.csv", source="local")
df_summary = load_file("summary.csv", source="local")

df_summary.head()

HTTPError: HTTP Error 404: Not Found

In [ ]:
output_types = [col.replace(' (%)', '') for col in df_summary.columns if col.endswith(' (%)')]
output_cols = [f"{col} (MW)" for col in output_types]
(df_summary[output_cols].iloc[-30*24:] / 1e3).plot()
plt.ylabel('GW')
plt.xlabel('Date')
plt.title(f'Ontario power generation by source for 30 days ending on {df_summary.index[-1].date().isoformat()}')
plt.legend([col.replace(' (MW)', '') for col in output_cols])
plt.savefig(
    os.path.join("..", "images", "ontario_power_generation_by_source_30_day.svg"),
    bbox_inches=bbox_inches,
    transparent=transparent,
    facecolor=facecolor,
)

In [ ]:
(df_summary[output_cols] / 1e3).plot()
plt.ylabel('GW')
plt.xlabel('Date')
plt.title(f'Ontario power generation by source ({df_summary.index[0].date().isoformat()} to {df_summary.index[-1].date().isoformat()})')
plt.legend([col.replace(' (MW)', '') for col in output_cols])
plt.savefig(
    os.path.join("..", "images", "ontario_power_generation_by_source.svg"),
    bbox_inches=bbox_inches,
    transparent=transparent,
    facecolor=facecolor,
)

In [ ]:
percentage_cols = [f"{col} (%)" for col in output_types]

df_summary[percentage_cols].plot()
plt.ylabel('%')
plt.xlabel('Date')
plt.title(f'Ontario power generation by source ({df_summary.index[0].date().isoformat()} to {df_summary.index[-1].date().isoformat()})')
plt.legend([col.replace(' (%)', '') for col in percentage_cols])
plt.savefig(
    os.path.join("..", "images", "ontario_power_generation_by_source_percentage.svg"),
    bbox_inches=bbox_inches,
    transparent=transparent,
    facecolor=facecolor,
)

In [ ]:
(df_summary['Ontario Demand (MW)'] / 1e3).plot()
plt.ylabel('GW')
plt.xlabel('Date')
plt.ylim([0, None])
plt.title(f'Ontario power demand ({df_summary.index[0].date().isoformat()} to {df_summary.index[-1].date().isoformat()})')
plt.savefig(
    os.path.join("..", "images", "ontario_power_demand.svg"),
    bbox_inches=bbox_inches,
    transparent=transparent,
    facecolor=facecolor,
)

In [ ]:
df_summary['CO2e Intensity (g/kWh)'].plot()
plt.ylabel('g/kWh')
plt.xlabel('Date')
plt.ylim([0, None])
plt.title(f'CO$_2$e intensity of the Ontario power grid ({df_summary.index[0].date().isoformat()} to {df_summary.index[-1].date().isoformat()})')
plt.savefig(
    os.path.join("..", "images", "ontario_co2_intensity.svg"),
    bbox_inches=bbox_inches,
    transparent=transparent,
    facecolor=facecolor,
)

In [ ]:
df_summary['CO2e Intensity (g/kWh)'].iloc[-24*30:].plot()
plt.ylabel('g/kWh')
plt.xlabel('Date')
plt.ylim([0, None])
plt.title(f'CO$_2$e intensity of the Ontario power grid for 30 days ending on {df_summary.index[-1].date().isoformat()}')
plt.savefig(
    os.path.join("..", "images", "ontario_co2_intensity_30_day.svg"),
    bbox_inches=bbox_inches,
    transparent=transparent,
    facecolor=facecolor,
)

In [ ]:
df_summary["month"] = df_summary.index.month
df_summary["year"] = df_summary.index.year
df_summary.groupby(["year", "month"]).sum()[output_cols].plot()
plt.ylabel('GWh')
plt.xlabel('(Year, Month)')
plt.title(f'Monthly Ontario power generation by source ({df_summary.index[0].date().isoformat()} to {df_summary.index[-1].date().isoformat()})')
plt.legend([col.replace(' (MW)', '') for col in output_cols])
plt.savefig(
    os.path.join("..", "images", "ontario_power_generation_by_source_monthly.svg"),
    bbox_inches=bbox_inches,
    transparent=transparent,
    facecolor=facecolor,
)